In [1]:
%cd /content/drive/MyDrive/AML/AML_course/Project

/content/drive/MyDrive/AML/AML_course/Project


In [2]:
from modules.textual_inversion import TextualInversionDataset, initialize_model, training_function, save_concept

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
what_to_teach = "object"
placeholder_token = f"<lol>"
# initializer_token = swapped_dict[worst_class]
initializer_token = "flower"
pretrained_model_name_or_path = "stabilityai/stable-diffusion-2"

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from diffusers import DDPMScheduler
import itertools
import torch

In [6]:
text_encoder, vae, unet, tokenizer = initialize_model(
    pretrained_model_name_or_path, placeholder_token, initializer_token
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer/tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [7]:
save_path = '/content/drive/MyDrive/AML/Project artifacts/raw_model_examples'

In [8]:
train_dataset = TextualInversionDataset(
      data_root=save_path,
      tokenizer=tokenizer,
      size=vae.sample_size,
      placeholder_token=placeholder_token,
      repeats=100,
      learnable_property=what_to_teach, #Option selected above between object and style
      center_crop=False,
      set="train",
)


<ipython-input-8-fb25fea1ebcb>:4: FutureWarning: Accessing config attribute `sample_size` directly via 'AutoencoderKL' object attribute is deprecated. Please access 'sample_size' over 'AutoencoderKL's config object instead, e.g. 'unet.config.sample_size'.
  size=vae.sample_size,


In [8]:
noise_scheduler = DDPMScheduler.from_config(pretrained_model_name_or_path, subfolder="scheduler")


hyperparameters = {
    "learning_rate": 5e-04,
    "scale_lr": True,
    "max_train_steps": 2000,
    "save_steps": 250,
    "train_batch_size": 4,
    "gradient_accumulation_steps": 1,
    "gradient_checkpointing": True,
    "mixed_precision": "fp16",
    "seed": 42,
    "output_dir": "sd-concept-output"
}
!mkdir -p sd-concept-output

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:245: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddpm.DDPMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mode

scheduler/scheduler_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [12]:
placeholder_token_id = tokenizer.encode(placeholder_token, add_special_tokens=False)[0]

In [14]:
import accelerate

accelerate.notebook_launcher(training_function, args=(text_encoder,
                                                      vae,
                                                      unet,
                                                      hyperparameters,
                                                      train_dataset,
                                                      noise_scheduler,
                                                      tokenizer,
                                                      placeholder_token_id,
                                                      pretrained_model_name_or_path))

for param in itertools.chain(unet.parameters(), text_encoder.parameters()):
  if param.grad is not None:
    del param.grad  # free some memory
  torch.cuda.empty_cache()



Launching training on one GPU.


  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


KeyboardInterrupt: 

In [6]:
save_concept_to_public_library = True
name_of_your_concept = "test"
hf_token_write = ""
path_with_images = "textual_inversion_images"
placeholder_token = "<flower>"
what_to_teach = "object"
pretrained_model_name_or_path = "stabilityai/stable-diffusion-2"
# save_path = "textual_inversion_images"
save_path = '/content/drive/MyDrive/AML/Project artifacts/raw_model_examples'

In [9]:
save_concept(
    hyperparameters=hyperparameters,
    path_with_images=save_path,
    name_of_your_concept= name_of_your_concept,
    placeholder_token = placeholder_token,
    what_to_teach = what_to_teach,
    pretrained_model_name_or_path = pretrained_model_name_or_path)

NameError: name 'output_dir' is not defined